In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

c:\Users\jebcu\anaconda3\envs\databricks\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
workplace_vector_db_path = "C:\\Users\\jebcu\\Desktop\\SYTYCH_Hackathon\\workplace_db"
 
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(collection_name="workplace_docs", embedding_function=hf_embed, persist_directory=workplace_vector_db_path)

Using embedded DuckDB with persistence: data will be stored in: C:\Users\jebcu\Desktop\SYTYCH_Hackathon\workplace_db


In [4]:
def get_similar_docs(question, similar_doc_count):
  return db.similarity_search(question, k=similar_doc_count)
 
# Let's test it with blackberries:
for doc in get_similar_docs("how to write a cover letter?", 2):
  print(doc.page_content)

Right now I am writing a cover letter to apply for a job, some people tell me cover letters should be short while other people tell me a cover letter should take up a page. Anyone who works in HR knows which one is the truth? Thanks


I don't work in HR but my opinion is that the cover letter should be to the point in terms of showing how great a fit you are for the position by citing specific examples so that you demonstrate that you believe you can do the job well.  The idea is to get the interview to further discuss specifics to validate the initial match.  While a cover letter may be a page on the high end if one has more than a handful of points to make, the key is to keep the fluff to a minimum in the cover letter and demonstrate how you fit this position so well.

I figured out I may as well send a .PDF file featuring my CV and Cover Letter. It sounds simpler to handle on the other side. The cover letter begins on a new page so when it's printed, it looks like 2 different docume

In [5]:
def build_qa_chain():
  torch.cuda.empty_cache()
  model_name = "databricks/dolly-v2-7b" # can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.
 
  # Increase max_new_tokens for a longer response
  # Other settings might give better results! Play around
  instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", 
                               return_full_text=True, max_new_tokens=256, top_p=0.95, top_k=50)
  # Note: if you use dolly 12B or smaller model but a GPU with less than 24GB RAM, use 8bit. This requires %pip install bitsandbytes
  # instruct_pipeline = pipeline(model=model_name, load_in_8bit=True, trust_remote_code=True, device_map="auto")
  # For GPUs without bfloat16 support, like the T4 or V100, use torch_dtype=torch.float16 below
  # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
 
  # Defining our prompt content.
  # langchain will load our similar documents as {context}
  template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
  Instruction: 
  You are applying for a job at a company. You need to respond to a question during the application process. 
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs. If you don't know, say that you do not know.
 
  {context}
 
  Question: {question}
 
  Response:
  """
  prompt = PromptTemplate(input_variables=['context', 'question'], template=template)
 
  hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
  # Set verbose=True to see the full prompt:
  return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)


In [6]:
# Building the chain will load Dolly and can take several minutes depending on the model size
qa_chain = build_qa_chain()

In [7]:
from IPython.display import display, HTML

In [8]:
def displayHTML(html):
  display(HTML(html))

In [9]:
def answer_question(question):
  similar_docs = get_similar_docs(question, similar_doc_count=2)
  result = qa_chain({"input_documents": similar_docs, "question": question})
  result_html = f"<p><blockquote style=\"font-size:24\">{question}</blockquote></p>"
  result_html += f"<p><blockquote style=\"font-size:18px\">{result['output_text']}</blockquote></p>"
  result_html += "<p><hr/></p>"
  for d in result["input_documents"]:
    source_id = d.metadata["source"]
    result_html += f"<p><blockquote>{d.page_content}<br/>(Source: <a href=\"https://workplace.stackexchange.com/a/{source_id}\">{source_id}</a>)</blockquote></p>"
  display(HTML(result_html))

In [10]:
answer_question("What are your weakenesses?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
  Instruction: 
  You are applying for a job at a company. You need to respond to a question during the application process. 
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs. If you don't know, say that you do not know.
 
  I have been trying to write this section of "Developmental Areas" in a employment form given by my organization. This is the place where I need to cite a few of my weaknesses.
I actually want to state that "I pay attention to detail in every aspect. (which is actually a positive point). But sometimes in doing so it tends to getting my work completed beyond normal time, meaning my over perfectionism tends to create issues of delay in project completion."
Now I would 

In [11]:
answer_question("Write a cover letter for a job application.")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
  Instruction: 
  You are applying for a job at a company. You need to respond to a question during the application process. 
  Use only information in the following paragraphs to answer the question at the end. Explain the answer with reference to these paragraphs. If you don't know, say that you do not know.
 
  Right now I am writing a cover letter to apply for a job, some people tell me cover letters should be short while other people tell me a cover letter should take up a page. Anyone who works in HR knows which one is the truth? Thanks


I don't work in HR but my opinion is that the cover letter should be to the point in terms of showing how great a fit you are for the position by citing specific examples so that you demonstrate that you believe you can do the job well